In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
import tensorflow as tf
import keras.backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model

from tensorflow.keras.layers import Input
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split

import cv2
import glob
import random

## 1. Data Prep

### Non-Generator

In [7]:
lst_fire_img = glob.glob('/Users/samhumphries/Dropbox/Sams_Sanctuary/School/15.095/Project/fire_dataset/fire_images/*.png')
lst_non_fire_img = glob.glob('/Users/samhumphries/Dropbox/Sams_Sanctuary/School/15.095/Project/fire_dataset/non_fire_images/*.png')

In [8]:
lst_fire = []
for x in lst_fire_img:
    lst_fire.append([x,1])
lst_nn_fire = []
for x in lst_non_fire_img:
    lst_nn_fire.append([x,0])
lst_complete = lst_fire + lst_nn_fire
random.shuffle(lst_complete)

df = pd.DataFrame(lst_complete,columns = ['files','target'])
df.head(10)

,files,target
0,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
1,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
2,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
3,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
4,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
5,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
6,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
7,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,0
8,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,1
9,/Users/samhumphries/Dropbox/Sams_Sanctuary/Sch...,0


In [10]:
filepath_img = '/kaggle/input/fire-dataset/fire_dataset/non_fire_images/non_fire.189.png'
df = df.loc[~(df.loc[:,'files'] == filepath_img),:]

In [11]:
def preprocessing_image(filepath):
    img= cv2.imread(filepath) #read
    img.shape
    img = cv2.cvtColor(img,cv2.COLOR_RGB2BGR) #convert
    img = cv2.resize(img,(256,256))  # resize
    img = img / 255 #scale
    return img 

def create_format_dataset(dataframe):
    X = []
    y = []
    for f,t in dataframe.values:
        try:
            X.append(preprocessing_image(f))
            y.append(t)
        except:
            pass
  
    return np.array(X),np.array(y)

In [12]:
X, y = create_format_dataset(df);
X.shape,y.shape

((998, 256, 256, 3), (998,))

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify = y)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((798, 256, 256, 3), (200, 256, 256, 3), (798,), (200,))

### Generator Way (*Ignore)

### Data Generators

## Model

In [16]:
model = tf.keras.Sequential()
model.add(Conv2D(filters=32,kernel_size = (2,2),activation='relu',input_shape = (256,256,3)))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 255, 255, 32)      416       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 126, 126, 64)      8256      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 62, 62, 128)       32896     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 31, 31, 128)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 123008)           

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stoppping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5)

In [18]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy',Recall(),AUC()])


#model.fit(train_gen, batch_size=32, epochs=2, validation_data=val_gen,callbacks=[early_stoppping,reduce_lr_on_plateau])
model.fit(X_train,y_train, batch_size=32, epochs=2, validation_data=(X_test,y_test),callbacks=[early_stoppping,reduce_lr_on_plateau])

Epoch 1/2
25/25 [==============================] - 27s 1s/step - loss: 0.7061 - accuracy: 0.7427 - recall: 0.8068 - auc: 0.7430 - val_loss: 0.1121 - val_accuracy: 0.9600 - val_recall: 0.9868 - val_auc: 0.9906
Epoch 2/2
25/25 [==============================] - 24s 937ms/step - loss: 0.0946 - accuracy: 0.9609 - recall: 0.9782 - auc: 0.9922 - val_loss: 0.0959 - val_accuracy: 0.9600 - val_recall: 0.9934 - val_auc: 0.9914


In [19]:
model.layers[7].get_weights()[0].shape

(123008, 64)

### Second Model

In [68]:
model2 = tf.keras.Sequential()
model2.add(Conv2D(filters=32,kernel_size = (2,2),activation='relu',input_shape = (256,256,3)))#,weights=model.layers[0].get_weights()))
model2.add(MaxPool2D())
model2.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))#,weights=model.layers[2].get_weights()))
model2.add(MaxPool2D())
model2.add(Conv2D(filters=128,kernel_size=(2,2),activation='relu'))#,weights=model.layers[4].get_weights()))
model2.add(MaxPool2D())
model2.add(Flatten())
model2.add(Dense(64,activation=None))#,weights=model.layers[7].get_weights()))

In [69]:
model2.layers[0].set_weights([model.layers[0].get_weights()[0],model.layers[0].get_weights()[1]])
model2.layers[2].set_weights([model.layers[2].get_weights()[0],model.layers[2].get_weights()[1]])
model2.layers[4].set_weights([model.layers[4].get_weights()[0],model.layers[4].get_weights()[1]])
model2.layers[7].set_weights([model.layers[7].get_weights()[0],model.layers[7].get_weights()[1]])

In [70]:
activations = model2.predict(X_train)

In [71]:
activations_test = model2.predict(X_test)

In [28]:
activations.shape

(798, 64)

In [50]:
model.layers[7].output

<KerasTensor: shape=(None, 64) dtype=float32 (created by layer 'dense')>

### Test on XGBoost

In [77]:
X_train_emb = activations
y_train_emb = y_train

X_test_emb = activations_test
y_test_emb = y_test

In [72]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.metrics import accuracy_score

In [78]:
# fit model no training data
XGBoost = xgb.XGBClassifier(objective = "binary:logistic", random_state=42)
XGBoost.fit(X_train_emb, y_train_emb)

[22:27:21] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/samhumphries/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=12,
              num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [79]:
y_pred = XGBoost.predict(X_test_emb)
predictions = [round(value) for value in y_pred]

In [80]:
accuracy = accuracy_score(y_test_emb, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 95.50%


### Logistic Regression

In [73]:
lr = LogisticRegression(max_iter = 300)
lr.fit(X_train_emb,y_train_emb)

LogisticRegression(max_iter=300)

In [74]:
y_pred=lr.predict(X_test_emb)
accuracy = accuracy_score(y_test_emb, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.00%


### CART

In [81]:
dt = tree.DecisionTreeClassifier()
dt.fit(X_train_emb,y_train_emb)

DecisionTreeClassifier()

In [82]:
y_pred=dt.predict(X_test_emb)
accuracy = accuracy_score(y_test_emb, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.00%


### Holistic Deep Learning

In [55]:
%cd "HolisticDL/src"
#%cd ..

/Users/samhumphries/Dropbox/Sams_Sanctuary/School/15.095/Project/HolisticDL/src


In [56]:
%run train.py --batch_range 32 --network_size 256 128 --stab_ratio_range 0.8 --l2 1e-5 --data_set activations --train_size 0.8 --lr 3e-4 --val_size 0.2

Instructions for updating:
non-resource variables are not supported in the long term
Batch Size: 32  ; stability subset ratio: 0.8  ; dropout value: 1


UnboundLocalError: local variable 'X_train' referenced before assignment

In [3]:
%run train.py --data_set uci10 --is_stable --stab_ratio_range 0.8

Instructions for updating:
non-resource variables are not supported in the long term
Batch Size: 64  ; stability subset ratio: 0.8  ; dropout value: 1
(1468, 21)
[0.43102831 0.43573095 0.43379643 0.43142807 0.42941474 0.43457447
 0.43534686 0.4326187  0.42777773 0.43763041 0.43379643 0.4326187
 0.47180438 0.46860346 0.47368115 0.47526599 0.46910821 0.46960847
 0.47180438 0.47059556 0.47180438]
[0.24659401 0.25476839 0.2513624  0.2472752  0.24386921 0.2527248
 0.25408719 0.2493188  0.24114441 0.25817439 0.2513624  0.2493188
 0.33446866 0.32561308 0.33991826 0.34468665 0.32697548 0.32833787
 0.33446866 0.33106267 0.33446866]
(940, 21)
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[-6.77944698e-17  3.57633545e-16 -5.05505803e-17  7.53534351e-17
  2.33383053e-16  2.11651028e-16  3.22791439e-16  8.24399650e-17
 -9.86208751e-17  2.77555756e-16 -4.67238541e-16 -5.66922396e-17
  9.40854959e-16  2.93854775e-16 -1.33935416e-16 -8.48021417e-17
 -5.52749336e-17 -8.07864414e-17 -